In [29]:
## 1. import libraries
import pandas as pd
import numpy as np

## 2. file path and load csv
# FOLDER_PATH = "/codes/" ## 파일 경로
# file_path = FOLDER_PATH+"lending_club_2020_train.csv"
file_path = "/Users/admin/codes/jupyter_codes/data/lending_club_2020_train.csv"
origin_data = pd.read_csv(file_path)
origin_data.head()

## 3. drop할 컬럼들을 일단 처리한다.
with open('drop_columns_0402.txt', mode='r') as f:
    drop_fields = f.readlines()
    drop_fields = [drop_field.strip('\n') for drop_field in drop_fields]
    
drop_df = origin_data.drop(columns=drop_fields)

# 컬럼 개수 출력
print(len(drop_df.columns)) # 96

## 4. 반응변수 처리
# loan_status가 "current", "issued", "policy" 인 행을 필터링하여 삭제
modified_df = drop_df[~drop_df['loan_status'].isin(['Current', 'Issued', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]

# risk = 1, safe = 0 으로 처리
modified_df.loc[modified_df['loan_status'].isin(['Fully Paid', 'In Grace Period']), 'loan_status'] = 0
modified_df.loc[modified_df['loan_status'].isin(['Charged Off', 'Default', 'Late (16-30 days)', 'Late (31-120 days)']), 'loan_status'] = 1

print(len(modified_df)) # 1133666


## 5. 데이터 처리용 함수
def delete_suffix(term:str)->int:
    '''첫 단어만을 저장하는 함수'''
    term = term.strip().split()[0]
    return int(term)

def delete_suffix_percentage(term:str)->float:
    '''%를 자르는 함수'''
    term = term.strip('%')
    return float(term)

def delete_suffix_xx(term:str)->int:
    '''xx를 자르는 함수'''
    term = term.strip('xx')
    return int(term)

def apply_to_multiple_columns(date_columns:list)->None:
    '''modified_df에서 %b-%Y(Aug-2006) 형식의 날짜 정보를 %Y-%m-%d(2006-08-01)형태로 저장하는 함수'''
    for col in date_columns:
        modified_df[col]=pd.to_datetime(modified_df[col], format='%b-%Y')

/var/folders/4k/lxd3n8m934z_82zsdzv73d5r0000gn/T/ipykernel_7763/767680825.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  origin_data = pd.read_csv(file_path)


77
1133666


In [30]:
## 1. import libraries
import pandas as pd
import numpy as np

## 2. file path and load csv
# FOLDER_PATH = "/codes/" ## 파일 경로
# file_path = FOLDER_PATH+"lending_club_2020_train.csv"
file_path = "data/filtered_lending_club_data.csv"
origin_data = pd.read_csv(file_path)
origin_data.head()
filtered_df = origin_data

In [31]:
filtered_df = filtered_df.dropna(subset=filtered_df.columns.difference(['loan_status']),how='all')

In [ ]:
from typing import Union
# 최빈값으로 대체하는 함수 작성
def fill_na_with_value(df, column_name:str, filling_value:Union[str, int])->None:
    '''
    df: dataframe to fill NA
    column_name : column name to change NA values
    filling_value : value type or just value to fill column's NA
    '''
    if filling_value=="mode":
        mode_value = df[column_name].mode()[0]
    elif filling_value=="median":
        mode_value = df[column_name].median()
    else:
        mode_value = filling_value
    df[column_name].fillna(mode_value, inplace=True)

In [ ]:
fill_na_with_value()

In [32]:
## 6. 결측 처리
# 6-1) 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
filtered_df = filtered_df.dropna(subset=['chargeoff_within_12_mths','collections_12_mths_ex_med','dti',
                                         'pub_rec_bankruptcies','revol_util'])
print(filtered_df.shape) # (1131682, 79)

# 6-2) 대체하기 어려운 변수 또는 결측이 많은 변수 삭제
print(filtered_df.shape) #(1131682, 72)

# 6-3) 결측 대체
# A. joint 관련 변수 : 결측 0 대체



# B. 최빈값 대체
filtered_df['mo_sin_old_il_acct'] = filtered_df['mo_sin_old_il_acct'].fillna(129)
filtered_df['mths_since_recent_bc'] = filtered_df['mths_since_recent_bc'].fillna(13)
filtered_df['mths_since_recent_inq'] = filtered_df['mths_since_recent_inq'].fillna(5)
filtered_df['bc_open_to_buy'] = filtered_df['bc_open_to_buy'].fillna(5200.0)
filtered_df['emp_length'] = filtered_df['emp_length'].fillna('10+ years')

# C. 2012년 대체
### 업데이트예정 ###


# D. 2015년 대체
### 업데이트 예정 ###



(1131682, 72)
(1131682, 72)


In [33]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression

In [40]:
y_train = filtered_df['loan_status']
X_train = filtered_df[['acc_now_delinq','annual_inc', 'delinq_2yrs', 'delinq_amnt','inq_last_6mths', 'loan_amnt',
'open_acc','pub_rec','revol_bal','term','total_acc']]

In [41]:
X_train.head()

,acc_now_delinq,annual_inc,delinq_2yrs,delinq_amnt,inq_last_6mths,loan_amnt,open_acc,pub_rec,revol_bal,term,total_acc
0,0.0,45000.0,1.0,0.0,0.0,6000.0,14.0,0.0,3090.0,36,36.0
1,0.0,110000.0,1.0,0.0,1.0,23200.0,24.0,0.0,16909.0,60,60.0
2,0.0,65000.0,0.0,0.0,0.0,16000.0,18.0,0.0,3881.0,36,28.0
3,0.0,50000.0,0.0,0.0,0.0,4500.0,7.0,2.0,6824.0,36,9.0
4,0.0,60000.0,0.0,0.0,0.0,20000.0,19.0,0.0,16626.0,36,40.0


In [42]:
y_train.head()

0    0
1    1
2    1
3    0
4    0
Name: loan_status, dtype: int64

In [43]:
estimator = LogisticRegression() # 학습시킬 모델 지정
selector = RFECV(estimator, step=1, cv = 5, min_features_to_select=20) # 한 step에 제거할 featrue 개수 및 cross validation fold 수 지정
selector = selector.fit(X_train, y_train)

In [44]:
selected_columns = X_train.columns[selector.support_]

selected_columns

Index(['acc_now_delinq', 'annual_inc', 'delinq_2yrs', 'delinq_amnt',
       'inq_last_6mths', 'loan_amnt', 'open_acc', 'pub_rec', 'revol_bal',
       'term', 'total_acc'],
      dtype='object')

In [45]:
selector.support_.sum()

11